In [36]:
import pandas as pd
import numpy as np
from numpy import mean
import os 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import numpy as np 
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import LinearSVC
from matplotlib import pyplot
import os

In [37]:
df = pd.read_csv("penguins_size.csv")

# Data Preprocessing

# drop na's
df = df.dropna()

# Binary encode categorical variables
df = pd.get_dummies(df, columns=['island', 'sex'])

# Split Predictor vs Target variables
X = df.loc[:, df.columns != 'species']
y = df.loc[:, df.columns == 'species']


col_names = X.columns
# Scale

X = pd.DataFrame(StandardScaler().fit_transform(X.values))

X = pd.DataFrame(X_scale, index=X_scale.index, columns=X_scale.columns)

X.set_axis(col_names, axis = 1, inplace = True)

X.head()


,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,island_Biscoe,island_Dream,island_Torgersen,sex_.,sex_FEMALE,sex_MALE
0,-0.897653,0.783487,-1.429521,-0.571229,-0.982194,-0.763504,2.47111,-0.0548,-0.988095,0.994030
1,-0.824290,0.121896,-1.072408,-0.509011,-0.982194,-0.763504,2.47111,-0.0548,1.012049,-1.006006
2,-0.677564,0.427246,-0.429605,-1.193405,-0.982194,-0.763504,2.47111,-0.0548,1.012049,-1.006006
3,-1.337831,1.088836,-0.572450,-0.944535,-0.982194,-0.763504,2.47111,-0.0548,1.012049,-1.006006
4,-0.860972,1.750427,-0.786718,-0.695664,-0.982194,-0.763504,2.47111,-0.0548,-0.988095,0.994030


In [38]:
cv = StratifiedKFold(n_splits=10)

### Neural Network Model 1

In [39]:
## default settings
neuralnet = MLPClassifier(solver='lbfgs', activation = 'logistic', alpha=0.0001,
                   hidden_layer_sizes=(100,), random_state=1)

nn_scores = cross_val_score(neuralnet, X, y, scoring = 'accuracy', cv=cv, n_jobs=-1, error_score='raise')


print(mean(nn_scores))

0.9969696969696968


### NN Model 2

In [40]:
## increase hidden layer size
neuralnet = MLPClassifier(solver='lbfgs', activation = 'logistic', alpha=0.0001,
                   hidden_layer_sizes=(500,), random_state=1)

nn_scores = cross_val_score(neuralnet, X, y, scoring = 'accuracy', cv=cv, n_jobs=-1, error_score='raise')


print(mean(nn_scores))

0.9939393939393938


### NN Model 3

In [41]:
neuralnet = MLPClassifier(solver='lbfgs', activation = 'logistic', alpha=0.0001,
                   hidden_layer_sizes=(3,), random_state=1)

nn_scores = cross_val_score(neuralnet, X, y, scoring = 'accuracy', cv=cv, n_jobs=-1, error_score='raise')


print(mean(nn_scores))

0.9939393939393938


### NN Model 4

In [52]:
## change activation function to 'tanh' and choose correspondig alpha based on highest accuracy
neuralnet = MLPClassifier(solver='lbfgs', activation = 'tanh', alpha=0.0001,
                   hidden_layer_sizes=(200,), random_state=1)

nn_scores = cross_val_score(neuralnet, X, y, scoring = 'accuracy', cv=cv, n_jobs=-1, error_score='raise')


print(mean(nn_scores))

0.9910873440285204


### NN Model 5

In [48]:
neuralnet = MLPClassifier(solver='lbfgs', activation = 'logistic', alpha=1, 
                          hidden_layer_sizes=(200,), random_state=1)

nn_scores = cross_val_score(neuralnet, X, y, scoring = 'accuracy', cv=cv, n_jobs=-1, error_score='raise')


print(mean(nn_scores))

0.9940285204991088


The highest performing model had an accuracy of 99.7%.

### Stacking Model

In [49]:
# split into training and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     X, y, random_state=42)

estimators = [
     ('svc', LinearSVC(random_state=42)),
     ('cart', DecisionTreeClassifier()),
     ('rf', RandomForestClassifier(min_samples_split = 25))
 ]

reg = StackingClassifier(
     estimators=estimators
)

In [50]:
accuracy = reg.fit(X_train, y_train).score(X_test, y_test)
print('The accuracy is: ',round(accuracy*100, 2),'%')

The accuracy is:  98.81 %


Compared to the neural network model, the stacking model performed worse in terms of accuracy. However, the stacking model still performed well with a 98.81% accuracy. 